In [155]:
# utils.py

import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import json
import time

# Function to get market data from yfinance
def get_chart(market, interval, now=datetime.now()):
    start = now - timedelta(days=29 if interval in ['1d', '1h', '15m', '5m'] else 364)
    df = yf.download(market, start=start, end=now, interval=interval).reset_index()
    if 'Datetime' in df.columns: df = df.rename(columns={"Datetime": "Date",})
    return df

# Trade Class
# from utils import get_chart

class Trade:
    def __init__(self, market, entry_date, entry_price, position_size, trade_type, interval):
        self.market = market
        self.entry_date = entry_date
        self.entry_price = entry_price
        self.position_size = position_size
        self.trade_type = trade_type
        self.interval = interval

        self.exit_date = None
        self.status = True # True for open, False for closed
        self.profit = None

    def __repr__(self):
        return f"""
        Trade: 
        market={self.market}, 
        entry_date={self.entry_date}, 
        entry_price={self.entry_price}, 
        trade_type={self.trade_type}, 
        exit_date={self.exit_date}, 
        status={self.status}, 
        profit={self.profit})
        """

    def __str__(self):
        return f"""
        Trade: 
        market={self.market}, 
        entry_date={self.entry_date}, 
        entry_price={self.entry_price}, 
        trade_type={self.trade_type}, 
        exit_date={self.exit_date}, 
        status={self.status}, 
        profit={self.profit})
        """
        # position_size={self.position_size}, 
    
    def close_trade(self, exit_date, exit_price):
        if self.status:
            self.exit_date = exit_date
            self.exit_price = exit_price
            if self.trade_type == 'buy':
                self.profit = (self.exit_price - self.entry_price)* self.position_size
            elif self.trade_type =='sell':
                self.profit = (self.entry_price - self.exit_price)* self.position_size
            self.status = False

# MarketEnv class
            
import random

class MarketEnv:
    def __init__(self, market, interval, current_index=0):
        self.market = market
        self.interval = interval
        self.current_index = current_index
        self.trades = []
        self.current_date = self._get_random_date(datetime(2022, 1, 1), datetime(2023, 12, 31))
        self.data = self._get_data(self.current_date)

    def _get_data(self, now):
        df = get_chart(self.market, self.interval, now=now)
        ## option to add indicator columns to dataframe
        df['EMA5'] = df['Close'].ewm(span=5, adjust=False).mean()
        df['EMA10'] = df['Close'].ewm(span=10, adjust=False).mean()
        df['EMA20'] = df['Close'].ewm(span=20, adjust=False).mean()
        return df

    def _get_random_date(self, start_date, end_date):
        delta = end_date - start_date
        random_days = random.randint(0, delta.days)
        random_date = start_date + timedelta(days=random_days)
        return random_date

    def simulate(self, strategy):
        for i in range(len(self.data)):
            data = self.data.loc[:i, strategy.columns]
            for index, row in data.iterrows():
                # print(index)
                self.current_date = row['Date']
                strategy.evaluate_market(row)
    

# Strategy class
            
class Strategy:
    def __init__(self, conditions, columns, marketenv):
        self.marketenv = marketenv
        self.market = marketenv.market
        self.interval = marketenv.interval
        self.data = marketenv.data
        self.columns = columns # columns of interest (open, close, volume, indicators, etc...)
        self.conditions = conditions # list of conditions as lambda functions

    def evaluate_market(self, row):

        ## If strategy condition(s) is met return true else false
        ## row = a row in an ohlc df with conditions

        for condition in self.conditions:
            for trade in self.marketenv.trades:
                if trade.status and condition['exit'](row):
                    trade.close_trade(row['Date'], row['Close'])

            if condition['enter'](row):
                entry_price = row['Close']
                trade = Trade(
                    self.market,
                    row['Date'],
                    entry_price,
                    1,
                    condition['type'],
                    self.interval
                )
                self.marketenv.trades.append(trade)
                return True

In [156]:
marketenv = MarketEnv('EURUSD=X', '1d')
conditions = [
    {
        'enter': lambda row: row['EMA5'] > row['EMA10'],
        'type': 'buy',
        'exit': lambda row: row['EMA5'] < row['EMA10'],
    },
]
strategy = Strategy(conditions, marketenv.data.columns, marketenv)
marketenv.simulate(strategy)

[*********************100%***********************]  1 of 1 completed


In [157]:
marketenv.trades

[
         Trade: 
         market=EURUSD=X, 
         entry_date=2023-03-28 00:00:00, 
         entry_price=1.0807071924209595, 
         trade_type=buy, 
         exit_date=None, 
         status=True, 
         profit=None)
         ,
 
         Trade: 
         market=EURUSD=X, 
         entry_date=2023-03-28 00:00:00, 
         entry_price=1.0807071924209595, 
         trade_type=buy, 
         exit_date=None, 
         status=True, 
         profit=None)
         ,
 
         Trade: 
         market=EURUSD=X, 
         entry_date=2023-03-29 00:00:00, 
         entry_price=1.0839520692825317, 
         trade_type=buy, 
         exit_date=None, 
         status=True, 
         profit=None)
         ,
 
         Trade: 
         market=EURUSD=X, 
         entry_date=2023-03-28 00:00:00, 
         entry_price=1.0807071924209595, 
         trade_type=buy, 
         exit_date=None, 
         status=True, 
         profit=None)
         ,
 
         Trade: 
         market=EURUSD=X, 
  